# Your First RAG Application: Personal Wellness Assistant

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application by building a **Personal Wellness Assistant**.

Imagine having an AI assistant that can answer your health and wellness questions based on a curated knowledge base - that's exactly what we'll build here. We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents (Loading our Wellness Knowledge Base)
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation (Building the Wellness Assistant)
  - 🚧 Activity #1: Enhance Your Wellness Assistant

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utilities

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [3]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [4]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using a comprehensive Health & Wellness Guide as our sample data. This content covers exercise, nutrition, sleep, stress management, and healthy habits - perfect for building a personal wellness assistant!

In [5]:
text_loader = TextFileLoader("data/HealthWellnessGuide.txt")
documents = text_loader.load_documents()
len(documents)

1

In [6]:
print(documents[0][:100])

The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into manageable sized chunks that retain the most relevant local context.

In [7]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

21

Let's take a look at some of the documents we've managed to split.

In [8]:
split_documents[0:1]

['The Personal Wellness Guide\nA Comprehensive Resource for Health and Well-being\n\nPART 1: EXERCISE AND MOVEMENT\n\nChapter 1: Understanding Exercise Basics\n\nExercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.\n\nThe four main types of exercise are aerobic (cardio), strength training, flexibility, and balance exercises. A well-rounded fitness routine includes all four types. Adults should aim for at least 150 minutes of moderate-intensity aerobic activity per week, along with muscle-strengthening activities on 2 or more days per week.\n\nChapter 2: Exercises for Common Problems\n\nLower Back Pain Relief\nLower back pain affects approximately 80% of adults at some point in their lives. Gentle stretching and strengthening exercises can help alleviate discomfort and prevent future

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [9]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this paper](https://arxiv.org/abs/2205.13147) for an answer to question #1.2!


##### ✅ Answer:
1. Yes. OpenAI’s Embeddings API allows you to specify a lower-dimensional embedding using the optional dimensions parameter at inference time. The model’s default embedding size is 1536. You can request a smaller vector (e.g., 512, 256, etc.). The model itself is not retrained — the output is truncated in a principled way
- Example (API request body):
{
  "model": "text-embedding-3-small",
  "input": "Your text here",
  "dimensions": 512
}

> This capability is supported by the v3 embedding models, as documented in the OpenAI API reference.

2. OpenAI uses Matryoshka Representation Learning (MRL). Matryoshka Representation Learning (MRL). The embedding model is trained so that prefixes of the full embedding vector are independently meaningful A 1536-dim embedding contains nested representations:
- First 256 dims → useful embedding
- First 512 dims → better embedding
- First 1024 dims → even better
- Full 1536 dims → best quality

> This is why truncating the vector using the dimensions parameter still preserves semantic quality.
> Key intuition: Like Matryoshka dolls, each smaller embedding lives inside the larger one and still works.


We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [10]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer:
1. Much Faster Throughput
- Embedding generation is I/O-bound (network + API latency).
- sync: waits for each API call to return
- async: sends many requests concurrently
- This dramatically reduces total runtime when embedding many documents.

2. Better Resource Utilization
- CPU stays productive while waiting on network responses
- Idle wait time is minimized
- Especially efficient for large-scale ingestion pipelines

3. Scales Better for Large Datasets
- When embedding:
    - Thousands of documents
    - Large corpora for RAG
    - Batch ingestion into vector databases
- async allows you to scale horizontally without increasing wall-clock time linearly.

4. Improved User Experience
- For interactive or production systems:
    - Faster responses
    - Lower perceived latency
    - Smoother pipelines (less “waiting” behavior)

5. Easier Rate-Limit Management
- With async, you can:
    - Control concurrency (e.g., semaphores)
    - Batch requests efficiently
    - Stay within API rate limits while maximizing throughput

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [11]:
vector_db.search_by_text("What exercises help with lower back pain?", k=3)

[(' Relief\nLower back pain affects approximately 80% of adults at some point in their lives. Gentle stretching and strengthening exercises can help alleviate discomfort and prevent future episodes.\n\nRecommended exercises for lower back pain include:\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles and raise shoulders off floor. Hold briefly, then lower. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening abs an

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicity's sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analogous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer:
- Use the seed parameter: Include the seed parameter in your API call and set it to any arbitrary integer (e.g., 42). Using the same seed value for repeated requests is the primary mechanism for the system to make a "best effort" to sample deterministically.
- Set temperature to 0: The temperature parameter controls the randomness of the output. A value of 0 (or a minuscule value close to zero, like 0.0000000000000000000001 for older models/APIs where 0 might be handled differently) will make the model's token selection more deterministic, always picking the highest probability token.
- Keep all other parameters constant: Every other parameter in your request (including top_p, max_tokens, prompt wording, system messages, etc.) must be exactly the same for you to get the same output.
- Pin the model version: Language models are updated by OpenAI periodically, which can impact reproducibility even with the same parameters. Use a specific model version (e.g., gpt-4o-2024-08-06 rather than gpt-4o) to ensure the exact same model weights and infrastructure are used. 
- Monitoring Reproducibility
    - Check the system_fingerprint: The API response includes a system_fingerprint field, which is an identifier for the current combination of model weights and backend configuration used by the servers. If this fingerprint changes between requests, even with the same seed, it indicates that an update on the backend may affect the consistency of the output. 
- Limitations
    - Determinism is not guaranteed: Even with all the above measures, OpenAI notes that perfect determinism is not guaranteed due to the inherent complexities and non-deterministic nature of modern computing and large language models.
    - Longer outputs increase variability: Requests with larger max_tokens values tend to be less deterministic. 

### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [12]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [13]:
print(response)

Hello! The best way to write a loop in Python depends on what you're trying to achieve, but generally, using a `for` loop is considered very Pythonic and clear, especially when iterating over a sequence like a list, tuple, or range.

For example, if you want to repeat an action 5 times, you could write:

```python
for i in range(5):
    print(f"Iteration {i}")
```

If you want to loop while a condition is true, you can use a `while` loop:

```python
count = 0
while count < 5:
    print(f"Count is {count}")
    count += 1
```

In most cases, `for` loops are preferred for their readability and simplicity. If you share more about your specific use case, I'd be happy to help tailor the loop to your needs! 😊


## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [14]:
RAG_SYSTEM_TEMPLATE = """You are a helpful personal wellness assistant that answers health and wellness questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't have information about that in my wellness knowledge base"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Include a gentle reminder that users should consult healthcare professionals for medical advice when appropriate
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [15]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [16]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What are some natural remedies for improving sleep quality?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

Response: Based on the provided context, some natural remedies for improving sleep quality include:

1. Cognitive Behavioral Therapy for Insomnia (CBT-I) — This therapy helps address the thoughts and behaviors that impair sleep (Source 1).

2. Relaxation techniques such as progressive muscle relaxation — which can reduce physical tension and promote sleep readiness (Sources 1 and 3).

3. Herbal teas like chamomile or valerian root — these are known for their calming effects and can support relaxation before bedtime (Source 1).

4. Magnesium supplements — which may aid in sleep, though it is important to consult a healthcare provider before use (Source 1).

5. Meditation — helps calm the mind and can improve sleep quality (Source 1).

Additional sleep hygiene practices that support natural sleep improvement include:
- Avoiding caffeine after 2 PM
- Exercising regularly but not too close to bedtime
- Limiting alcohol and heavy meals before bed
- Maintaining a consistent sleep schedule
- 

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer:

###### Key Prompting Strategy

####### Chain-of-Thought Prompting

- This strategy encourages the model to reason step by step before arriving at a final answer.
- How it works
    - You explicitly instruct the model to:
        - Break down the problem
        - Think through intermediate steps
        - Explain reasoning in a logical sequence

######### Example prompt
>Explain your reasoning step by step before giving the final answer.
######### Or more softly (preferred in practice):
>Think through the problem carefully and explain your reasoning.

- Why This Works
    - The model allocates more tokens to reasoning
    - Reduces shallow or rushed answers
    - Improves accuracy on complex or multi-step tasks
    - Leads to clearer, more structured explanations

>Variants & Related Strategies
1. Instructional Reasoning Prompts
- Guide the reasoning structure without explicitly exposing internal logic:
    >Break the problem into smaller parts and address each one.

2. Few-Shot Reasoning
- Provide examples that demonstrate thoughtful reasoning:
- Q: Example problem  
    >A: Step-by-step reasoning → Final answer

3. Self-Consistency Prompting
- Ask the model to evaluate or refine its own answer:
    >Generate an answer, then review it for completeness and correctness.

4. Role-Based Reasoning
- Assign a role that implies depth and rigor:
    >You are a senior researcher. Provide a detailed, well-reasoned explanation.

### 🏗️ Activity #1:

Enhance your Personal Wellness Assistant in some way! 

Suggestions are: 

- **PDF Support**: Allow it to ingest wellness PDFs (meal plans, workout guides, medical information sheets)
- **New Distance Metric**: Implement a different similarity measure - does it improve retrieval for health queries?
- **Metadata Support**: Add metadata like topic categories (exercise, nutrition, sleep) or difficulty levels to help filter results
- **Different Embedding Model**: Try a different embedding model - does domain-specific tuning help for health content?
- **Multi-Source Ingestion**: Add the capability to ingest content from YouTube health videos, podcasts, or health blogs

While these are suggestions, you should feel free to make whatever augmentations you desire! Think about what features would make this wellness assistant most useful for your personal health journey.

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

## Implement new distance metrics and evaluate the responses
Extended the VectorDatabase class in aimakerspace/vectordatabase.py by adding three new distance metric functions alongside the existing cosine_similarity:

1. Cosine Similarity (Existing)
- Formula: cos(θ) = (A · B) / (||A|| × ||B||)
- Range: -1 to +1
- Interpretation: Measures the angle between vectors, ignoring magnitude

2. Euclidean Distance (New)
- Formula: √(Σ(aᵢ - bᵢ)²)
- Range: 0 to ∞ (returned as negative: -∞ to 0)
- Interpretation: Straight-line distance in n-dimensional space

3. Dot Product Similarity (New)
- Formula: Σ(aᵢ × bᵢ)
- Range: For normalized vectors: -1 to +1
- Interpretation: Sum of element-wise products; measures alignment

4. Manhattan Distance (New)
- Formula: Σ|aᵢ - bᵢ|
- Range: 0 to ∞ (returned as negative: -∞ to 0)
- Interpretation: "City block" distance - sum of absolute differences

> Why Negative Values for Distance Metrics?
The VectorDatabase.search() method sorts results in descending order (reverse=True) because higher similarity = better match. For distance metrics where lower = closer, you return the negative value so that:
- Closest vectors (small distance) → larger negative → sorted first
- Farthest vectors (large distance) → smaller negative → sorted last

> Evaluation Framework
Built a comparison system to test all metrics side-by-side:

##### Define test queries

In [17]:
### YOUR CODE HERE
test_queries = [
    {
        "query": "What exercises help with lower back pain?",
        "expected_topic": "exercise"
    },
    {
        "query": "How much water should I drink daily?",
        "expected_topic": "nutrition"  
    },
    {
        "query": "Tips for better sleep quality?",
        "expected_topic": "sleep"
    },
]

##### Create a comparison function

In [18]:
from aimakerspace.vectordatabase import (
    cosine_similarity,
    euclidean_distance,
    dot_product_similarity,
    manhattan_distance,
)

def compare_distance_metrics(query: str, vector_db, k: int = 3):
    """Compare retrieval results across different distance metrics."""
    
    metrics = {
        "cosine": cosine_similarity,
        "euclidean": euclidean_distance,
        "dot_product": dot_product_similarity,
        "manhattan": manhattan_distance,
    }
    
    results = {}
    for name, metric_fn in metrics.items():
        # Call vector_db.search_by_text with this metric
        result = vector_db.search_by_text(query, k=k, distance_measure=metric_fn)
        # Store the results
        results[name] = result
    
    return results

def display_metric_comparison(query: str, results: dict):
    """Display side-by-side comparison of retrieval results."""
    
    print(f"\n{'='*80}")
    print(f"QUERY: {query}")
    print(f"{'='*80}")
    
    for metric_name, result_list in results.items():
        print(f"\n--- {metric_name.upper()} ---")
        # result_list is a list of (text, score) tuples
        for i, (text, score) in enumerate(result_list, 1):
            # Show first 100 chars of text + the score
            preview = text[:100].replace('\n', ' ')  # Remove newlines for cleaner display
            print(f"  [{i}] Score: {score:.4f}")
            print(f"      {preview}...")
    
    

#### Run against different distance metrics and analyze output

In [19]:
   for test in test_queries:
       #print(f"\n{'='*60}")
       #print(f"Query: {test['query']}")
       results = compare_distance_metrics(test['query'], vector_db, k=3)
       # Display results somehow
       display_metric_comparison(test['query'], results)


QUERY: What exercises help with lower back pain?

--- COSINE ---
  [1] Score: 0.7478
       Relief Lower back pain affects approximately 80% of adults at some point in their lives. Gentle str...
  [2] Score: 0.4573
      The Personal Wellness Guide A Comprehensive Resource for Health and Well-being  PART 1: EXERCISE AND...
  [3] Score: 0.4353
      chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs. - Pelvic Tilts: L...

--- EUCLIDEAN ---
  [1] Score: -0.7086
       Relief Lower back pain affects approximately 80% of adults at some point in their lives. Gentle str...
  [2] Score: -1.0427
      The Personal Wellness Guide A Comprehensive Resource for Health and Well-being  PART 1: EXERCISE AND...
  [3] Score: -1.0639
      chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs. - Pelvic Tilts: L...

--- DOT_PRODUCT ---
  [1] Score: 0.7477
       Relief Lower back pain affects approximately 80% of adults at some point in their li

#### Results Interpretation
> Query 1: "What exercises help with lower back pain?"
- Cosine	0.7478	75% similarity - strong match
- Dot Product	0.7490	~75% - nearly identical to cosine
- Euclidean	-0.7102	Distance 0.71 - very close
- Manhattan	-21.9291	Distance 21.9 across 1536 dims
- Top retrieved chunk: "Lower back pain affects approximately 80% of adults..." ✅ Highly relevant

> Query 2: "How much water should I drink daily?"
- Cosine	0.4671	47% similarity - moderate match
- Dot Product	0.4671	Identical to cosine
- Euclidean	-1.0324	Distance 1.03 - moderately close
- Manhattan	-32.0749	Larger distance than Query 1
- Top retrieved chunk: "drink 8 glasses (64 ounces) of water daily..." ✅ Highly relevant

> Query 3: "Tips for better sleep quality?"
- Metric	Top Result Score	Interpretation
- Cosine	0.5713	57% similarity - good match
- Dot Product	0.5712	~57% - nearly identical
- Euclidean	-0.9261	Distance 0.93
- Manhattan	-28.8918	Moderate distance
- Top retrieved chunk: "Sleep occurs in cycles of about 90 minutes..." ✅ Relevant

##### Key Findings
1. All Metrics Produce Identical Rankings
- For all 3 queries, every distance metric retrieved the same chunks in the same order. This happens because:
    - OpenAI's text-embedding-3-small returns normalized vectors (||v|| ≈ 1)
    - For normalized vectors, the metrics are mathematically related: cosine_similarity ≈ dot_product (since ||A|| × ||B|| ≈ 1)
    - euclidean_distance² = 2 × (1 - cosine_similarity)
2. Cosine ≈ Dot Product for Normalized Embeddings
Query	Cosine	Dot Product	Difference
1	0.7478	0.7490	0.0012
2	0.4671	0.4671	0.0000
3	0.5713	0.5712	0.0001
The tiny differences are due to floating-point precision.

3. Score Scales Vary Significantly
Metric	Score, Range,	Best Value
Cosine,	0 to 1,	1 (identical)
Dot Product,	0 to 1 (normalized),	1 (identical)
Euclidean,	-√2 to 0 (normalized),	0 (identical)
Manhattan,	-∞ to 0,	0 (identical)
Manhattan has the largest absolute values because it sums differences across all 1536 dimensions.

4. Retrieval Quality is Excellent
- All queries retrieved highly relevant chunks as the top result, demonstrating that the RAG system works well regardless of which distance metric is used.

>Practical Implications
- For OpenAI embeddings: Cosine similarity and dot product are interchangeable - use whichever is computationally convenient
- Dot product is fastest: No normalization step needed at query time
- When metrics might differ: With non-normalized embeddings (some open-source models), you'd see ranking differences between cosine (angle-based) and Euclidean (magnitude-sensitive)
- Manhattan distance: Useful for sparse vectors or when you want to penalize outlier dimensions less severely than Euclidean

#### Enhancement #2: YouTube Transcript Integration

Adding the ability to ingest content from YouTube health videos.

In [20]:
from aimakerspace.youtube_utils import YouTubeTranscriptLoader

# Load transcript from the YouTube video
youtube_url = "https://www.youtube.com/watch?v=nexjJqMq3YM"
youtube_loader = YouTubeTranscriptLoader(youtube_url)
youtube_docs = youtube_loader.load_documents()

print(f"Loaded transcript with {len(youtube_docs[0])} characters")
print(f"Preview: {youtube_docs[0][:500]}...")

Loaded transcript with 29712 characters
Preview: Welcome back to season three
of the Fittr podcast. In last week's episode,
we spoke with Jitendra Chouksey aka JC the founder of FITTR,
where he spoke about his concept
of what it truly means to be healthy. It's not just about physical fitness,
but also mental, emotional, social
and being financially fit. And yet so many people struggle with this. They don't understand what they need to do
and why they can't achieve their goals. And that's exactly what we're going to
talk about in this episode o...


##### Chunk and combine documents

In [21]:
# Split YouTube transcript into chunks
youtube_chunks = text_splitter.split_texts(youtube_docs)
print(f"YouTube video split into {len(youtube_chunks)} chunks")

# Combine with existing documents
all_documents = split_documents + youtube_chunks
print(f"Original wellness guide: {len(split_documents)} chunks")
print(f"YouTube transcript: {len(youtube_chunks)} chunks")  
print(f"Total combined: {len(all_documents)} chunks")

YouTube video split into 38 chunks
Original wellness guide: 21 chunks
YouTube transcript: 38 chunks
Total combined: 59 chunks


##### Build combined vector database for youtube video and original text

In [22]:
# Build combined vector database
combined_vector_db = VectorDatabase()
combined_vector_db = asyncio.run(combined_vector_db.abuild_from_list(all_documents))
print(f"Combined vector database built with {len(combined_vector_db.vectors)} vectors")

Combined vector database built with 59 vectors


##### Test with query rekevant to youtube video

In [26]:
# Test query that might retrieve YouTube content
test_query = "How is labeling food good or bad not going to work?"

print("Results from ORIGINAL vector_db (text only):")
original_results = vector_db.search_by_text(test_query, k=3)
for i, (text, score) in enumerate(original_results, 1):
    preview = text[:1000].replace('\n', ' ')
    print(f"  [{i}] Score: {score:.4f} - {preview}...")

print("\nResults from COMBINED vector_db (text + YouTube):")
combined_results = combined_vector_db.search_by_text(test_query, k=3)
for i, (text, score) in enumerate(combined_results, 1):
    preview = text[:1000].replace('\n', ' ')
    print(f"  [{i}] Score: {score:.4f} - {preview}...")

Results from ORIGINAL vector_db (text only):
  [1] Score: 0.2473 - ls of Healthy Eating  A balanced diet provides your body with the nutrients it needs to function properly. The key components include:  Macronutrients: - Carbohydrates: Primary energy source. Choose complex carbs like whole grains, fruits, and vegetables. - Proteins: Essential for muscle repair and immune function. Good sources include lean meats, fish, eggs, legumes, and dairy. - Fats: Necessary for hormone production and nutrient absorption. Focus on healthy fats from olive oil, avocados, nuts, and fatty fish.  Micronutrients: - Vitamins: Organic compounds needed in small amounts. Eat a variety of colorful fruits and vegetables. - Minerals: Inorganic elements like calcium, iron, and zinc. Found in whole foods, dairy, and leafy greens.  Chapter 5: Meal Planning for Wellness  Effective meal planning saves time, reduces stress, and helps maintain a healthy diet.  Weekly Meal Planning Steps: 1. Review your schedule for th